In [102]:
import pandas as pd
from gensim import corpora, models
from gensim.corpora import Dictionary
from scipy.sparse import csr_matrix
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import keras as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import tensorflow as tf
import pymongo

In [103]:
client = pymongo.MongoClient(host="192.168.0.106", port=29197)
client["github"].authenticate("github", "git332", "github")
db = client["github"]

In [104]:
lang_set = [x["name"] for x in db.project_lang_30.find()]
npm_tag_set = ["npm_"+x["name"] for x in db.npm_tag_stem_top100.find()]
pkg_tag_set = ["pkg_"+x["name"] for x in db.composer_tag_stem_top100.find()]
pypi_tag_set = ["pypi_"+x["name"] for x in db.pypi_tag_stem_top100.find()]

In [105]:
train_col = ["file_npm", "file_pypi", "file_composer"]
train_col.extend(lang_set)
train_col.extend(npm_tag_set)
train_col.extend(pkg_tag_set)
train_col.extend(pypi_tag_set)
len(train_col)

333

In [106]:
lda_dim = 256

In [107]:
tag_set = [x["name"] for x in db.project_tag_more_than_100.find()]
len(tag_set)

116

In [108]:
lda = models.ldamodel.LdaModel.load('../model/readme_lda_{}.model'.format(lda_dim))

In [109]:
data_raw = pd.read_csv("../data/data_raw_tagonehot.csv", index_col=0)
data_raw.head(4)

,project_link,file_readme,file_npm,file_pypi,file_composer,JavaScript,HTML,CSS,Python,Shell,...,pytorch,cach,laravel,data-sci,natural-language-process,authent,computer-vis,compos,python3,yii2
0,/freeCodeCamp/freeCodeCamp,welcom freecodecamp org s open sourc codebas c...,1,0,0,0.941868,0.017145,0.039971,0.000000,0.001017,...,0,0,0,0,0,0,0,0,0,0
1,/vuejs/vue,support vue jsvue js mit licens open sourc pro...,1,0,0,0.976735,0.006227,0.003945,0.000000,0.001170,...,0,0,0,0,0,0,0,0,0,0
2,/twbs/bootstrap,bootstrap sleek intuit power end framework fas...,1,0,0,0.494735,0.182466,0.320497,0.000000,0.001512,...,0,0,0,0,0,0,0,0,0,0
3,/facebook/react,react middot react javascript librari build us...,1,0,0,0.951378,0.017036,0.003504,0.000079,0.001854,...,0,0,0,0,0,0,0,0,0,0


In [110]:
data_tag = np.array(data_raw[tag_set]).astype(np.int64)

In [111]:
data_minus_readme = data_raw[train_col]
data_minus_readme.shape

(25467, 333)

In [112]:
readme_list = list(data_raw.file_readme)
corpus = []

for post in readme_list:
    
    corpus.append([word for word in str(post).strip().lower().split()])

len(readme_list)

25467

In [113]:
dictionary = Dictionary(corpus)
doc_bow = dictionary.doc2bow(corpus[0])
lda_list = lda[doc_bow]

In [114]:
corpus_lda = [lda[dictionary.doc2bow(doc)] for doc in corpus]

In [115]:
data_matrix = []

for post in corpus_lda:
    arr = np.zeros(lda_dim)
    for elem in post:
        arr[elem[0]] = elem[1]
    
    data_matrix.append(arr)

In [116]:
data_x_np = np.array(data_matrix)
data_x_np.shape

(25467, 256)

In [117]:
data_full_feature = np.concatenate((data_x_np,data_minus_readme),axis=1)

In [118]:
data_full_feature.shape[1]

589

In [384]:
data_minus_readme.shape

(25467, 333)

In [385]:
len(data_x_np[1])

256

In [386]:
len(tag_set)

116

In [387]:
train_x, test_x, train_y, test_y = train_test_split(data_minus_readme, data_tag, train_size=0.75, test_size=0.25, random_state=1)

In [389]:
train_x.shape, test_x.shape

((19100, 333), (6367, 333))

In [400]:
import keras as K
from keras.layers.advanced_activations import PReLU, LeakyReLU, Softmax
input_dim = train_x.shape[1]
lr = 0.0001
init = K.initializers.glorot_uniform(seed=1)
simple_adam = K.optimizers.RMSprop(lr=lr)
model = K.models.Sequential()
act = K.layers.advanced_activations.PReLU()

model.add(K.layers.Dense(units=600, input_dim=input_dim, kernel_initializer=init,activation=LeakyReLU(0.7)))
# model.add(act)
# model.add(K.layers.Dense(units=600, kernel_initializer=init, activation='relu'))
# model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(units=400, kernel_initializer=init,activation=LeakyReLU(0.7)))
# model.add(act)
# model.add(K.layers.BatchNormalization())

model.add(K.layers.Dense(units=200, kernel_initializer=init,activation=LeakyReLU(0.7)))
# model.add(act)
# model.add(K.layers.BatchNormalization())

model.add(K.layers.Dense(units=116, kernel_initializer=init,activation=LeakyReLU(0.7)))
model.compile(loss='mean_squared_error', optimizer=simple_adam, metrics=['top_k_categorical_accuracy', 'categorical_accuracy'])

In [401]:
b_size = 256
max_epochs = 70
print("Starting training ")
h = model.fit(train_x, train_y, batch_size=b_size, epochs=max_epochs, shuffle=True, verbose=1)
print("Training finished \n")

Starting training 
Epoch 1/70
19100/19100 [==============================] - 2s 121us/step - loss: 0.0216 - top_k_categorical_accuracy: 0.5223 - categorical_accuracy: 0.2285
Epoch 2/70
19100/19100 [==============================] - 1s 51us/step - loss: 0.0160 - top_k_categorical_accuracy: 0.5940 - categorical_accuracy: 0.2099
Epoch 3/70
19100/19100 [==============================] - 1s 51us/step - loss: 0.0152 - top_k_categorical_accuracy: 0.6048 - categorical_accuracy: 0.2121
Epoch 4/70
19100/19100 [==============================] - 1s 52us/step - loss: 0.0149 - top_k_categorical_accuracy: 0.6113 - categorical_accuracy: 0.2082
Epoch 5/70
19100/19100 [==============================] - 1s 51us/step - loss: 0.0146 - top_k_categorical_accuracy: 0.6248 - categorical_accuracy: 0.2194
Epoch 6/70
19100/19100 [==============================] - 1s 51us/step - loss: 0.0145 - top_k_categorical_accuracy: 0.6300 - categorical_accuracy: 0.2232
Epoch 7/70
19100/19100 [==============================] 

In [402]:
p = model.predict(test_x)

In [403]:
p[0],test_y[0]

(array([ 3.99190187e-02, -6.88938573e-02,  4.97753732e-02,  1.58765778e-01,
         2.10351571e-02, -1.17918672e-02, -1.39034195e-02,  2.05532275e-03,
        -1.55963534e-02, -2.55220151e-03, -3.25947627e-02, -1.82408486e-02,
         8.68331432e-01,  8.63281727e-01,  8.71645033e-01,  8.51837754e-01,
         8.45768631e-01, -1.38831278e-02, -2.26300228e-02,  6.70319330e-03,
         1.10439807e-02, -4.11782647e-03,  8.25692993e-03, -2.43066382e-02,
        -1.09302253e-02,  2.15638801e-03, -2.42224745e-02, -5.72637655e-03,
        -9.68197640e-03, -8.12165439e-03, -1.68814007e-02,  7.84499943e-02,
        -1.24975313e-02, -1.57340486e-02, -8.68360046e-03, -7.38475006e-03,
        -1.71911586e-02, -9.29899421e-03, -2.87130177e-02, -8.40720069e-03,
        -1.59693472e-02,  1.57315191e-02,  2.24599503e-02,  1.26017239e-02,
        -3.15353423e-02,  2.08640769e-02, -2.58164257e-02,  2.06209384e-02,
        -1.32190185e-02, -2.77736839e-02, -1.21625047e-02, -4.86988621e-03,
        -3.7

In [404]:
# 取出分数不为0的项
p_list = []
for k in p:
    p_tag = {}
    for i, score in enumerate(k.tolist()):
        if(score!=0):
            p_tag[score] = i
    p_list.append(p_tag)

In [405]:
def recall(true_tag, pre_tag):
    t = 0
    for i in pre_tag:
        if(i in true_tag):
            t += 1
    return t/len(true_tag)

def precision(true_tag, pre_tag):
    t = 0
    for i in pre_tag:
        if(i in true_tag):
            t += 1
    if len(pre_tag) == 0:
        return 1
    else:
        return t/len(pre_tag)

In [406]:
p_list_top5 = []
for i in range(len(p_list)):
    p_list_top5.append([p_list[i][k] for k in sorted(p_list[i].keys())][-5:])
    
p_list_top10 = []
for i in range(len(p_list)):
    p_list_top10.append([p_list[i][k] for k in sorted(p_list[i].keys())][-10:])
    
p_list_top116 = []
for i in range(len(p_list)):
    p_list_top116.append([p_list[i][k] for k in sorted(p_list[i].keys())][-116:])

In [407]:
np.array([0,0,2]).nonzero()

(array([2]),)

In [408]:
true_tag_real = []
for row in test_y:
    true_tag_real.append(row.nonzero()[0])

In [409]:
recal_at_116_count = 0
recal_at_10_count = 0
recal_at_5_count = 0
precision_at_116_count = 0
precision_at_10_count = 0
precision_at_5_count = 0

for i in range(len(true_tag_real)):
    recal_at_10_count += recall(true_tag_real[i], p_list_top10[i])


for i in range(len(true_tag_real)):
    recal_at_5_count += recall(true_tag_real[i], p_list_top5[i])
    
for i in range(len(true_tag_real)):
    recal_at_116_count += recall(true_tag_real[i], p_list_top116[i])
    
for i in range(len(true_tag_real)):
    precision_at_10_count += precision(true_tag_real[i], p_list_top10[i])    

for i in range(len(true_tag_real)):
    precision_at_5_count += precision(true_tag_real[i], p_list_top5[i])
    
for i in range(len(true_tag_real)):
    precision_at_116_count += precision(true_tag_real[i], p_list_top116[i])
    
recall_at_116 = recal_at_116_count/len(true_tag_real)
recall_at_10 = recal_at_10_count/len(true_tag_real)
recall_at_5 = recal_at_5_count/len(true_tag_real)
precision_at_116 = precision_at_116_count/len(true_tag_real)
precision_at_10 = precision_at_10_count/len(true_tag_real)
precision_at_5 = precision_at_5_count/len(true_tag_real)


print("lr: {} b_size: {} max_epochs: {} lda_dim: {} ".format(lr, b_size, max_epochs, lda_dim))

model.summary()
print()
print("====================================")
print("Recall@116: {}".format(recall_at_116))
print("Precision@116: {}".format(precision_at_116_count/len(true_tag_real)))
print("F1-score@116: {}".format((2*recall_at_116*precision_at_10)/(recall_at_116+precision_at_10)))
print()
print("Recall@10: {}".format(recall_at_10))
print("Precision@10: {}".format(precision_at_10_count/len(true_tag_real)))
print("F1-score@10: {}".format((2*recall_at_10*precision_at_10)/(recall_at_10+precision_at_10)))
print()
print("Recall@5: {}".format(recall_at_5))
print("Precision@5: {}".format(precision_at_5_count/len(true_tag_real)))
print("F1-score@5: {}".format((2*recall_at_5*precision_at_5)/(recall_at_5+precision_at_5)))
print("====================================")

lr: 0.0001 b_size: 256 max_epochs: 70 lda_dim: 256 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_142 (Dense)            (None, 600)               200400    
_________________________________________________________________
dense_143 (Dense)            (None, 400)               240400    
_________________________________________________________________
dense_144 (Dense)            (None, 200)               80200     
_________________________________________________________________
dense_145 (Dense)            (None, 116)               23316     
Total params: 544,316
Trainable params: 544,316
Non-trainable params: 0
_________________________________________________________________

Recall@116: 1.0
Precision@116: 0.02803788933238909
F1-score@116: 0.43599159901250806

Recall@10: 0.7974957467536397
Precision@10: 0.27876550965917407
F1-score@10: 0.41312331363813487

Recall@5: 0.6421309145629875
Pr